Проект e-commerce: вариант 1 
Продакт-менеджер Василий попросил вас проанализировать совершенные покупки и ответить на следующие вопросы:

1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)

3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)

4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут. (10 баллов)

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут. (35 баллов)

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных.

Файлы:

 
     olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей

customer_id — позаказный идентификатор пользователя

customer_unique_id —  уникальный идентификатор пользователя  (аналог номера паспорта)

customer_zip_code_prefix —  почтовый индекс пользователя

customer_city —  город доставки пользователя

customer_state —  штат доставки пользователя


    olist_orders_dataset.csv —  таблица заказов

order_id —  уникальный идентификатор заказа (номер чека)

customer_id —  позаказный идентификатор пользователя

order_status —  статус заказа

order_purchase_timestamp —  время создания заказа

order_approved_at —  время подтверждения оплаты заказа

order_delivered_carrier_date —  время передачи заказа в логистическую службу

order_delivered_customer_date —  время доставки заказа

order_estimated_delivery_date —  обещанная дата доставки


    olist_order_items_dataset.csv —  товарные позиции, входящие в заказы

order_id —  уникальный идентификатор заказа (номер чека)

order_item_id —  идентификатор товара внутри одного заказа

product_id —  ид товара (аналог штрихкода)

seller_id — ид производителя товара

shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике

price —  цена за единицу товара

freight_value —  вес товара

— Пример структуры данных можно визуализировать по order_id == 00143d0f86d6fbd9f9b38ab440ac16f5

Уникальные статусы заказов в таблице olist_orders_dataset:

created —  создан

approved —  подтверждён

invoiced —  выставлен счёт

processing —  в процессе сборки заказа

shipped —  отгружен со склада

delivered —  доставлен пользователю

unavailable —  недоступен

canceled —  отменён



Подключим необходимые библиотеки и настроим параметры графиков 

In [ ]:
import pandas               as pd
import numpy                as np
import seaborn              as sns
import matplotlib.pyplot    as plt
from datetime     import timedelta
from operator    import attrgetter
import warnings
warnings.filterwarnings('ignore')
import requests 
from urllib.parse import urlencode
from datetime     import timedelta

In [ ]:
sns.set(
    font_scale=1,
    style="whitegrid",
    rc={'figure.figsize':(40,7)}
        )

Прочитаем данные которые нам предоставили 

In [ ]:
df_c = pd.read_csv\
('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-nikitin/1_FIRST_PROJECT_E_COM/olist_customers_dataset.csv')

In [ ]:
df_c.head()
# таблица с уникальными идентификаторами пользователей

In [ ]:
df_o = pd.read_csv\
('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-nikitin/1_FIRST_PROJECT_E_COM/olist_orders_dataset.csv')

In [ ]:
df_o.head()
# таблица заказов

In [ ]:
df_i = pd.read_csv\
('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-nikitin/1_FIRST_PROJECT_E_COM/olist_order_items_dataset.csv')

In [ ]:
df_i.head()
# товарные позиции, входящие в заказы

Проверим типы данных 

In [ ]:
df_c.dtypes

In [ ]:
df_o.dtypes

In [ ]:
df_i.dtypes

Проверим размер датафреймов

In [ ]:
df_c.shape

In [ ]:
df_o.shape

In [ ]:
df_i.shape

Проверим количество нулевых значений

In [ ]:
df_c.isna().sum()

In [ ]:
df_o.isna().sum()

In [ ]:
df_i.isna().sum()

Сколько уникальных значений встречается

In [ ]:
df_c.nunique()

In [ ]:
df_o.nunique()

In [ ]:
df_i.nunique()

In [ ]:
df_c.groupby('customer_id').agg({'customer_id':'count'})

# 1. Сколько у нас пользователей, которые совершили покупку только один раз?


In [ ]:
df_o['customer_id'].describe()

In [ ]:
df_o['order_status'].describe()

In [ ]:
df_o['order_status'].value_counts()
# посмотрев на основные показания, необходимо убрать для очистки данных все нулевые строки 

In [ ]:
df_o.groupby("order_status", as_index=False).agg({'customer_id':'count'})

предлагаю в качестве покупки считать только те заказы которые имеют статус (delivered)

In [ ]:
df_o_q=df_o.query("order_status == 'delivered'")

теперь объеденим таблицы df_o и df_c(пользователи и заказы) с помощью метода (merge)

In [ ]:
df_o_q1=df_o_q.merge(df_c, how='left', on='customer_id')

In [ ]:
df_o_q1\
    .groupby('customer_unique_id', as_index=False)\
    .agg({'order_id':'count'})\
    .query("order_id == 1")\


90557

# Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? 

Так как вопрос касается только заказов, будем работать с таблицей заказов df_o

In [ ]:
df_o.head()

Так как нам нужно найти причины того, что заказы не доставлены, проверим какие статусы заказов у нас имееются и их количество 

In [ ]:
df_o.order_status.value_counts()

Далее проверим типы данных

In [ ]:
df_o.dtypes

Так как у нас стоит задача дать ответ по месяцам, переведем нужные нам колонки в формат даты

In [ ]:
df_o['order_purchase_timestamp']=pd.to_datetime(df_o['order_purchase_timestamp'])

In [ ]:
df_o['order_approved_at']=pd.to_datetime(df_o['order_approved_at'])

In [ ]:
df_o['order_delivered_carrier_date']=pd.to_datetime(df_o['order_delivered_carrier_date'])

In [ ]:
df_o['order_delivered_customer_date']=pd.to_datetime(df_o['order_delivered_customer_date'])

In [ ]:
df_o['order_estimated_delivery_date']=pd.to_datetime(df_o['order_estimated_delivery_date'])

In [ ]:
df_o.dtypes

In [ ]:
df_o.head()

Итак

approved - заказ подтвержден, но еще не доставлен

canceled - заказ отменен, о доставке нет речи

created - заказ только создан, еще далеко до доставки

delivered - доставлен пользователю, здесь явно ошибка, заказы с таким статусом нам не подойдут

invoiced - выставлен счёт, пока рано доставлять

processing - в процессе сборки заказа, еще не доставили

shipped - отгружен со склада, еще в процессе доставки

unavailable - недоступен, не доставлен

У нас есть статус delivered, что противоречит условию, убираем заказы с этим статусом

Для дальнейшего анализа проверим данные на наличие Nan 

In [ ]:
df_o.isna().sum()

Перезапишем датафрейм и заменим Nan значения на 0

In [ ]:
df_o.fillna(0).query('order_delivered_customer_date == 0')\
    .groupby('order_status',as_index=False)\
    .agg({'order_purchase_timestamp':'count'})\
    .rename(columns={'order_purchase_timestamp':'count'})

In [ ]:
undelivered=df_o.fillna(0).query('order_delivered_customer_date == 0 and order_status != "delivered"')

In [ ]:
undelivered.head()

In [ ]:
undelivered.isna().sum()

In [ ]:
undelivered.shape[0]

2957
Получается такое количество заказов не доставлено, теперь нужно найти среднее количество недоставленных заказов по месяцам

Приведем нужную нам колонку к месяцу

In [ ]:
undelivered['order_purchase_timestamp'] = pd.to_datetime(undelivered.order_purchase_timestamp)
undelivered['order_purchase_timestamp'] = undelivered.order_purchase_timestamp.dt.strftime('%Y-%m')

In [ ]:
undelivered

Теперь ответим на вопрос о причинах 

In [ ]:
reasons=undelivered.groupby(['order_status','order_purchase_timestamp'], as_index=False)\
    .agg({'order_id':'count'})\
    .groupby('order_status', as_index=False)\
    .agg({'order_id':'mean'})\
    .round()\
    .sort_values(by='order_id',ascending=False)

In [ ]:
reasons

Визуализируем причины

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x='order_status', y='order_id', data=reasons)

plt.title(label = 'Not delivered  orders',
          color = 'black',
          fontsize = 20,
          x = 0.5,
          y = 1.1,
          family = 'serif')

plt.show()

###  Нетрудно заметить, что наибольшее количество заказов, которые не доставили находятся на этапе shipped(отгружен со склада) 
###  Возможно есть проблемы с оптимизацией передачи заказа логистической фирме(партнеру), а может и поменять партнера доствки. 

# По каждому товару определить, в какой день недели товар чаще всего покупается. 

Так как нам предстоит работать с товарами, объеденим две таблицы df_o и df_i(таблица заказов и таблица с товварами входящими в заказ)

In [ ]:
df_o_i=df_o.merge(df_i)

In [ ]:
df_o_i.head()

**Теперь применим условие о покупках, напомню что за покупки я взял только те заказы у которых есть статус "delivered"**

In [ ]:
order_items=df_o_i.query("order_status == 'delivered'")

In [ ]:
order_items.head()

Проверим типы данных 

In [ ]:
order_items.dtypes

Создам колонку в которой будет название дня недели(по колонке "order_delivered_customer_date", так как покупки я определяю по статусу "delivered")

In [ ]:
order_items['day']=order_items.order_delivered_customer_date.dt.day_name()

Ответим на вопрос задачи: 

In [ ]:
order_items.groupby(['product_id', 'day'], as_index=False)\
    .agg({'order_id':'count'})\
    .sort_values('order_id', ascending=False)\
    .drop_duplicates(subset='product_id')


# Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.

Объеденим все три таблицы, взяв одну объединённую из прошлого задания, где в качестве покупки я беру статус заказа "delivered"

In [ ]:
merge_orders_items_customers=order_items.merge(df_c)

In [ ]:
merge_orders_items_customers.dtypes

In [ ]:
merge_orders_items_customers.head()

Необходимо найти покупки в месяц, поэтому поработаем с order_purchase_timestamp и приведем данные столбца к месяцу

In [ ]:
merge_orders_items_customers['order_delivered_customer_date'] = merge_orders_items_customers\
.order_delivered_customer_date.dt.strftime('%Y-%m')

Создадим новый датафрейм, где укажем количество покупок по месяцам

In [ ]:
df_e=merge_orders_items_customers.groupby(['customer_unique_id', 'order_delivered_customer_date'], as_index=False)\
    .agg({'order_status':'count'})

Создадим новую колонку с количеством дней в месяце

In [ ]:
df_e['day']=pd.to_datetime(df_e.order_delivered_customer_date)

In [ ]:
df_e.day = df_e.day.dt.days_in_month

Теперь добавим колонку с количеством недель в месяце для более точного значения в метрике 

In [ ]:
df_e['weeks']=df_e.day/7

In [ ]:
df_e.head()

Теперь найдем количество покупок по месяцам 

In [ ]:
df_e['metric']=df_e.order_status/df_e.weeks

In [ ]:
df_e.head()

# Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.

Для начала объеденим таблицы df_o и df_i(таблица заказов и таблица с товварами входящими в заказ)

In [ ]:
for_kohort=df_o.merge(df_c)

In [ ]:
for_kohort.head()

По условию задачи нам нужно работать с retention.

Retention (Возвратность) — характеристика группы пользователей, вычисляемая соотношением активных пользователей в определённые временные промежутки. Пример: некоторое приложение установили 50 человек, через неделю активными пользователями остались лишь 5 человек; возвратность: 5 / 50 * 100% = 10%.

Для начала уберем все нулевые значения

In [ ]:
for_kohort.isna().sum()

In [ ]:
for_kohort_new = for_kohort.dropna(axis='index', how='any', subset=['order_approved_at'])

In [ ]:
for_kohort_new.isna().sum()

Теперь надо найти дату первой покупки 

In [ ]:
for_kohort_new.dtypes

In [ ]:
for_kohort_new['first_purchase'] = for_kohort_new.groupby('customer_unique_id')['order_approved_at'].transform('min')


In [178]:
for_kohort_new.order_purchase_timestamp = for_kohort_new.order_purchase_timestamp.dt.strftime('%Y-%m')
for_kohort_new.order_approved_at = for_kohort_new.order_approved_at.dt.strftime('%Y-%m')
for_kohort_new.first_purchase = for_kohort_new['first_purchase'].dt.strftime('%Y-%m')


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
for_kohort_new.head()

Найдем число уникальных покупателей в период с января по декабрь.

Для чего отберем только те строки, где статус заказа говорит о покупке, выберем строки, где первая покупка была сделана в промежуток с января по декабрь, сгруппируем по датам покупки и посчитаем число уникальных покупателей.

In [179]:
for_kohort_table = for_kohort_new.query("order_status ==  'delivered'")\
    .query("'2017-01' <= first_purchase <= '2017-12'")\
    .groupby(['first_purchase', 'order_approved_at'],as_index=False).agg({'customer_unique_id': 'nunique'})


Приведем даты к номеру месяца

In [180]:
for_kohort_table.first_purchase = pd.to_datetime(for_kohort_table.first_purchase).dt.to_period("m")
for_kohort_table.order_approved_at = pd.to_datetime(for_kohort_table.order_approved_at).dt.to_period("m")


Посчитаем число месяцев с первой покупки до последующего подтвержения заказа


In [181]:
for_kohort_table['diff'] = (for_kohort_table.order_approved_at - for_kohort_table.first_purchase)\
    .apply(attrgetter("n"))

In [182]:
for_kohort_table_finish = for_kohort_table\
    .pivot(index='first_purchase', columns = 'diff', values = 'customer_unique_id')


In [183]:
retention = for_kohort_table_finish.div(for_kohort_table_finish.iloc[:,0], axis = 0)
final_table = (retention
            .style
            .set_caption('Retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту

final_table

diff,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
first_purchase,,,,,,,,,,,,,,,,,,,
2017-01,100.00%,0.59%,0.29%,0.15%,0.44%,0.15%,0.44%,0.15%,0.15%,,0.29%,0.29%,0.73%,0.44%,0.15%,0.15%,0.29%,0.44%,0.15%
2017-02,100.00%,0.25%,0.25%,0.19%,0.43%,0.12%,0.25%,0.19%,0.12%,0.25%,0.12%,0.31%,0.19%,0.19%,0.12%,0.06%,0.06%,0.25%,
2017-03,100.00%,0.52%,0.36%,0.36%,0.36%,0.16%,0.16%,0.32%,0.32%,0.12%,0.36%,0.12%,0.24%,0.12%,0.16%,0.24%,0.08%,0.16%,
2017-04,100.00%,0.63%,0.22%,0.18%,0.27%,0.27%,0.36%,0.27%,0.36%,0.18%,0.22%,0.09%,0.09%,0.04%,0.09%,0.09%,0.22%,,
2017-05,100.00%,0.55%,0.52%,0.38%,0.32%,0.35%,0.43%,0.17%,0.26%,0.26%,0.29%,0.35%,0.20%,0.06%,0.20%,0.26%,,,
2017-06,100.00%,0.49%,0.36%,0.39%,0.26%,0.33%,0.39%,0.23%,0.13%,0.23%,0.33%,0.36%,0.16%,0.13%,0.20%,,,,
2017-07,100.00%,0.54%,0.35%,0.22%,0.35%,0.19%,0.32%,0.11%,0.19%,0.24%,0.24%,0.30%,0.13%,0.24%,,,,,
2017-08,100.00%,0.69%,0.37%,0.27%,0.37%,0.51%,0.27%,0.27%,0.12%,0.17%,0.25%,0.17%,0.12%,,,,,,
2017-09,100.00%,0.67%,0.50%,0.32%,0.42%,0.27%,0.22%,0.22%,0.32%,0.17%,0.27%,0.07%,,,,,,,


В период с января по декабрь необходимо было найти когорту с самым высоким retention на 3й месяц.

Ответим на вопрос задачи:

In [184]:
retention[2].idxmax()

Period('2017-05', 'M')

# Часто для качественного анализа аудитории использую подходы, основанные на сегментации.

## Используя python,построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики:

### R - время от последней покупки пользователя до текущей даты,

### F - суммарное количество покупок у пользователя за всё время,

### M - сумма покупок за всё время.

### Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.

Для начала объединим все три таблицы в одну и сбросим пустые значения

In [185]:
full_table = df_o_i.merge(df_c).dropna()  

In [186]:
full_table.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP


В качестве текущей примем дату самой последней в столбце order_approved_at, увеличенную на 1 день, будто анализируем на следующий день после сбора данных

In [187]:
yesterday = full_table.order_delivered_customer_date.max()

In [188]:
today = yesterday + timedelta(days=1)


Теперь подготовим данные для RFM

In [189]:
for_rfm = full_table.groupby('customer_unique_id',as_index=False)\
                    .agg({'order_delivered_customer_date': lambda x: (today - x.max()).days,\
                          'product_id': lambda x: len(x),\
                          'price': lambda x: x.sum()})\
                    .rename(columns={'order_delivered_customer_date': 'R', 'product_id': 'F', 'price': 'M'})

Когда имеются значения для RFM, нужно выделить диапазоны

Каждый квантиль будет содержать 20%

Разбиваем на диапазон: до 20%, 20-40%, 40-60%, 60-80% и выше 80 %

In [190]:
for_rfm.head()

,customer_unique_id,R,F,M
0,0000366f3b9a7992bf8c76cfdf3221e2,154,1,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,160,1,18.90
2,0000f46a3911fa3c0805444483337064,560,1,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,350,1,25.99
4,0004aac84e0df4da2b147fca70cf8255,324,1,180.00


In [191]:
quantiles = for_rfm.quantile(q=[.2, .4, .6, .8])
quantiles

,R,F,M
0.2,128.0,1.0,39.9
0.4,209.0,1.0,69.9
0.6,301.0,1.0,109.9
0.8,419.0,1.0,179.9


Выделим сегменты покупателей

In [192]:
def RClass(value,parameter,quantiles_table):
    if value <= quantiles_table[parameter][0.20]:
        return 1
    elif value <= quantiles_table[parameter][0.40]:
        return 2
    elif value <= quantiles_table[parameter][0.60]:
        return 3
    elif value <= quantiles_table[parameter][0.80]:
        return 4
    else:
        return 5
    
    
def FMClass(value,parameter,quantiles_table):
    if value <= quantiles[parameter][0.20]:
        return 5
    elif value <= quantiles[parameter][0.40]:
        return 4
    elif value <= quantiles[parameter][0.60]:
        return 3
    elif value <= quantiles_table[parameter][0.80]:
        return 2
    else:
        return 1   

Применяем функцию для каждого кластера

In [193]:
for_rfm['R_range'] = for_rfm['R'].apply(RClass, args=('R',quantiles))

for_rfm['F_range'] = for_rfm['F'].apply(FMClass, args=('F',quantiles))

for_rfm['M_range'] = for_rfm['M'].apply(FMClass, args=('M',quantiles))

for_rfm['RFM_r'] = for_rfm.R_range.map(str) + for_rfm.F_range.map(str) + for_rfm.M_range.map(str)


In [194]:
for_rfm

,customer_unique_id,R,F,M,R_range,F_range,M_range,RFM_r
0,0000366f3b9a7992bf8c76cfdf3221e2,154,1,129.90,2,5,2,252
1,0000b849f77a49e4a4ce2b2a4ca5be3f,160,1,18.90,2,5,5,255
2,0000f46a3911fa3c0805444483337064,560,1,69.00,5,5,4,554
3,0000f6ccb0745a6a4b88665a16c9f078,350,1,25.99,4,5,5,455
4,0004aac84e0df4da2b147fca70cf8255,324,1,180.00,4,5,1,451
...,...,...,...,...,...,...,...,...
93337,fffcf5a5ff07b0908bd4e2dbc735a684,469,2,1570.00,5,1,1,511
93338,fffea47cd6d3cc0a88bd621562a9d061,281,1,64.89,3,5,4,354
93339,ffff371b4d645b6ecea244b27531430a,603,1,89.90,5,5,3,553
93340,ffff5962728ec6157033ef9805bacc48,157,1,115.00,2,5,2,252


Таким образом, выделили сегменты, где 111 - сегмент покупателей, у которых наименьшее время с последней покупки, наибольшее число заказов и сумма покупок, а 555 - сегмент покупателей, у которых наибольшее время с последней покупки, наименьшее число заказов и сумма покупок.

In [206]:
for_rfm.groupby("RFM_r").agg({'customer_unique_id':'count'})


,customer_unique_id
RFM_r,
111,955
112,534
113,407
114,248
115,141
151,2767
152,3398
153,3030
154,3509
